## Defining Functions

In [3]:
import requests
import base64
import pyarrow as pa
import time


def to_arrow_table(byte_string: str) -> pa.Table:
  """Get a raw base64 string and convert to an Arrow Table."""
  with pa.ipc.open_stream(base64.b64decode(byte_string)) as reader:
    return pa.Table.from_batches(reader, reader.schema)

def get_sl_results_back(graph_ql_query, env_id, dbt_cloud_sl_token):

  # Any additional headers you need to send, such as authentication tokens.
  headers = {
      "Authorization": f"Bearer {dbt_cloud_sl_token}"
  }

  # make a post
  gql_resp = requests.post(
    "https://semantic-layer.cloud.getdbt.com/api/graphql",
    json={"query": graph_ql_query},
    headers=headers
  )

  # get query id
  query_id = gql_resp.json()['data']['createQuery']['queryId']


  # poll for results
  polling_query = f"""
        {{
          query(environmentId: {env_id}, queryId: "{query_id}") {{
            sql
            status
            error
            totalPages
            jsonResult
            arrowResult
          }}
        }}
      """

  # sleep for 3 seconds to give the query some buffer time
  time.sleep(3)

  # get the response
  query_complete = False

  # waiting for the repsonse to complete before getting the answers
  while query_complete == False:

    gql_response_table = requests.post(
      "https://semantic-layer.cloud.getdbt.com/api/graphql",
      json={"query": polling_query},
      headers=headers,
    )

    query_reponse = gql_response_table.json()['data']["query"]

    if query_reponse == None:

      time.sleep(5)

    else:

       query_complete = True

  # turn results into table
  arrow_table = to_arrow_table(query_reponse["arrowResult"]).to_pandas()

  # return results
  return arrow_table

# Hitting the sementic Layer GraphQL APIs to get a metric spilt by a dimension

In [4]:
# defining the dbt cloud SL token
dbt_cloud_sl_token = "<<your token here>>"

# defining SL env deal
env_id = 123456

# defining the query I want to send down
send_query = f"""
  mutation {{
    createQuery(
      environmentId: {env_id}
      metrics: [{{name: "order_total"}}]
      groupBy: [{{name: "customer__customer_state"}}]
    ) {{
      queryId
    }}
  }}
"""


# sending and getting results
sl_query_results = get_sl_results_back(send_query, env_id, dbt_cloud_sl_token)


# display results
display(sl_query_results)

,customer__customer_state,order_total
0,MS,493374
1,CA,1374350
2,MO,659505
3,NC,724664
4,KS,295189
5,NE,507348
6,CO,410203
7,HI,199317
8,LA,194105
9,NY,271160


# End of script